## Advanced topics
The following will dedicate for some second-order (Newton) optimization methods or related topics which require the second order derivative information.

We define some mathematical notations including $\mathbf{H}$ for Hessian matrix, $\mathbf{G}$ for Gauss-Newton Hessain matrix. 

Suppose we are trying to optimize
$$\min_x f(x)$$
At a step $k+1$ of a gradient descent, the update will be done as
$$x^{(k+1)} = x^{(k)} - \eta \times \nabla f(x^{(k)})$$
This is called first-order optimization since we only consider first-order derivative information. Second-order optimization, on the other hand, does not require to set learning rate $\eta$ but precomputed based on the second-order derivative. One can use the following update
$$x^{(k+1)} = x^{(k)} - \mathbf{H}^{-1} \nabla f(x^{(k)})$$
or
$$x^{(k+1)} = x^{(k)} - \mathbf{G}^{-1} \nabla f(x^{(k)})$$

Now, the computational challenge would lie in how to efficiently obtain the inverse of Hessian which is often expensive to compute. One of the common resorts is to use iterative algorithms. The approaches includes Conjugate Gradient (CG), Neumann series approximation and Knonecker Factorization Approximation Curvature (KFAC).

**Conjugate Gradient** Instead of finding $\mathbf{H}^{-1} \nabla f(x^{(k)})$, we're going to solve an optimization problem which will result in a same value:
$$\min_a a^\top\mathbf{H}a + (\nabla f)^\top a$$

Finding inverse Hessian using such methods can be seen at some work such as 

1. [Meta-Learning with Implicit Gradients](https://arxiv.org/abs/1909.04630)

However, one important assumption that Conjugate Gradient makes is that $\mathbf{H}$ or $\mathbf{G}$ is possitive definite. This is not always true for Hessian $\mathbf{H}$. 

**Neumann series**
With a certain assumption (maximum spectral value less than 1), we can have the following series expansion of an inverse matrix
$$A^{-1} = \sum_{i=0}^\infty (I - A)^i$$

1. [Second-Order Stochastic Optimization for Machine Learning in Linear Time](https://arxiv.org/abs/1602.03943)

2. [Understanding Black-box Predictions via Influence Functions](https://arxiv.org/abs/1703.04730)

3. [Optimizing Millions of Hyperparmeters by Implicit Differentiation](https://arxiv.org/abs/1911.02590)

**KFAC** Inspired by graphical approximation, Fisher information is estimated with structure breakdown in neural works where layers become independent in the approximation. This relaxation is convenient for inversion operation and Knonecker product becomes handy. The reference consists of
1. [Optimizing Neural Networks with Kronecker-factored Approximate Curvature](https://arxiv.org/abs/1503.05671)

The following notebook tries to cover such methods.

In [2]:
import torch
import torch.nn as nn
from torch.autograd import grad
from second import *

In [3]:
def conjugate_gradient(mvp, vector, parameters, cg_iter=10, eps=0.):
    """
    Conjugate gradient method
    Reference: https://en.wikipedia.org/wiki/Conjugate_gradient_method
    Arguments:
        mvp: matrix vector product as a callable function. In this case, it corresponds to Hessian-vector product
        vector: a vector. In this case, it is the gradient vector
        parameters: parameters
    """
    x = vector.clone().detach()
    # residual
    r = vector.clone().detach() - mvp(x)
    p = r.clone().detach()
    for _ in range(cg_iter):
        Ap = mvp(p)
        alpha = (r @ r) / (p @ Ap + eps)
        x = x + alpha * p
        r_new = r - alpha * Ap
        beta = (r_new @ r_new) / (r @ r + eps)
        p = r_new + beta * p
        r = r_new.clone().detach()
    return x

Create a simple model and compute a loss value

In [4]:
input_dim = 3
output_dim = 1
model = nn.Sequential(
    nn.Linear(input_dim, 5),
    nn.Sigmoid(),
    nn.Linear(5, output_dim)
)

x = torch.randn(10, input_dim)
y = torch.randn(10, output_dim)
criterion = nn.MSELoss()


Define the MVP as the Hessian-vector product as one input of ```conjugate_gradient``` method

In [5]:
def cg_mvp(vector):
    return hvp(loss, tuple(model.parameters()), vector)
    # return gvp(loss, logit, tuple(model.parameters()), vector)

In [6]:
logit = model(x)
loss = criterion(logit, y)
gradient = grad(loss,tuple(model.parameters()), retain_graph=True)
cg_inv_hvp = conjugate_gradient(cg_mvp, vector=parameters_to_vector(gradient), parameters=tuple(model.parameters()), cg_iter=40)

Now, let's see what the exact computation looks like (of course the following should be avoided in practice)

In [7]:
def exact_hessian(loss, parameters):
    gradient = grad(
        loss,
        parameters,
        create_graph=True,
        retain_graph=True
    )
    gradient = parameters_to_vector(gradient)
    hessian = torch.zeros((gradient.numel(), gradient.numel()))
    for i in range(gradient.numel()):
        H_i = grad(
            gradient[i],
            parameters,
            retain_graph=True
        )
        H_i = parameters_to_vector(H_i)
        hessian[i] = H_i
    return hessian

In [8]:
logit = model(x)
loss = criterion(logit, y)
hessian = exact_hessian(loss, tuple(model.parameters()))

In [9]:
exact_inv_hvp = torch.inverse(hessian) @ parameters_to_vector(gradient)

In [30]:
values, _= torch.eig(hessian)
torch.max(values)

tensor(4.3236)

Compute the error

In [10]:
relative_error = torch.norm(cg_inv_hvp - exact_inv_hvp) / torch.norm(exact_inv_hvp)
relative_error

tensor(0.6924)

In [11]:
cosine_similarity = cg_inv_hvp @ exact_inv_hvp / (torch.norm(cg_inv_hvp) * torch.norm(exact_inv_hvp))
cosine_similarity

tensor(0.7375)

We can see that the approximation method (CG here) return a good estimate based on the cosine similarity

In [12]:
def neumann(mvp, parameters, vector, truncate_iter=10):
    p = v = vector
    for _ in range(truncate_iter):
        p = v + p - mvp(p)
    return p

In [13]:
nm_inv_hvp= neumann(cg_mvp, tuple(model.parameters()), vector=parameters_to_vector(gradient))

In [65]:
def neuman_this_case(parameters, vector, truncate_iter=10):
    v = vector
    p = v.clone().detach()
    for i in range(truncate_iter):
        x_i = x[i]
        y_i = y[i]
        output_i = model(x_i)
        loss_i = criterion(output_i, y_i)
        p = v + p - hvp(loss, parameters, p)/4.5
    
    return p

nm_inv_hvp = neuman_this_case(tuple(model.parameters()), vector=parameters_to_vector(gradient))
nm_inv_hvp
    

tensor([ 0.0752,  0.0680, -0.3132, -0.2627,  0.3481,  0.3965, -0.0778,  0.1506,
         0.1076, -0.1436,  0.1586,  0.2219, -0.2498,  0.2461,  0.4386, -0.3126,
        -0.2587, -0.1872, -0.1226, -0.0814, -1.1904, -0.5912, -0.3178, -0.1558,
         0.0867, -0.9277])

In [62]:
cosine_similarity = nm_inv_hvp @ exact_inv_hvp / (torch.norm(nm_inv_hvp) * torch.norm(exact_inv_hvp))

In [63]:
cosine_similarity

tensor(0.0067)